## Chargement des fichiers

In [17]:
import pandas as pd
import re
import requests
import os
from ipywidgets import IntProgress
from IPython.display import display
from pymarc import parse_xml_to_array
#import jellyfish

pd.options.mode.chained_assignment = None  # Remove copy in place warning for dataframe operations


# Définition du fichier d'alignement à vérifier
input_file_path = "input/test-dataset.tsv"

# Définition du fichier de sortie pour les cas suspects
output_file_root = "output/test-output"

# Faut-il exporter tous les alignements vérifiés avec une colonne identifiant les cas suspects (True),
# ou exporter ces derniers à part (False)?
output_in_place = True

# Types d'alignement à vérifier
valid_types = {'auto'}

# Types de validation à effectuer
validations = {'idref-local'}
# Les valeurs suivantes sont possibles:
# - 'dates' -> compare les dates présentes dans les formes principales source et cible (ne nécessite que les fichiers d'export Ouali)
# - 'auteur-titre' ATTENTION cette validation fait un appel d'API IdRef pour CHAQUE ALIGNEMENT. Utiliser uniquement lorsque nécessaire.
# - 'idref-local' -> compare les sous-champs des notices source et cible. Nécessite l'accès à un export des notices IdRef et ATC/RNV

# Définition des fichiers exportés de IdRef et ATC/RNV pour comparaison des sous-champs. Utilisé uniquement en mode 'idref-local'
idref_set = 'personnes'
idref_records_folder = "/Users/thomas/Documents/tmp-nobackup/bcu-rnv/idref-harvest-personnes"
atc_records_file_path = "input/20240917_all_ATC_persons_with_AD.tsv"

# Pour limiter le chargement des fichiers IdRef (ex. pour tester), renseigner cette variable pour ne charger que les x premier fichiers.
# Pour ne pas limiter, assigner la valeur -1
idref_records_limit = -1

# Fichiers temporaires pour accélérer le traitement successif lors de la comparaison des sous-champs. Si existants, ils sont utilisés
# en mode 'idref-local' en lieu et place des notices complètes ci-dessus.
idref_records_extract = "input/idref-extrait-" + idref_set + "-comparaison.csv"


# Nom de la colonne ajoutée
output_colname = "validation auto"

# Définition des colonnes contenant les formes sources et cible à comparer
source_form_colname = "forme principale source"
target_form_colname = "forme principale cible"
align_type_colname = "décision d'alignement"
target_colname = "réservoir cible"
target_candidatenr_colname = "nombre de candidats"
idref_id_colname = "id cible"
atc_id_colname = "id source"

# Le fichier à vérifier est chargé dans une dataframe
df = pd.read_csv(input_file_path, sep='\t', dtype = str)
print(df.shape)

# Pour la comparaison, on retire les types d'alignements ignorés ainsi que les non-alignements
df_filtered = df[(df[align_type_colname].isin(valid_types)) & (df[target_colname] == 'idref')]
df_rest = df[(~df[align_type_colname].isin(valid_types)) | (df[target_colname] != 'idref')]
print(df_filtered.shape)
print(df_rest.shape)
display(df_filtered.head())

df_filtered.reset_index(drop=True, inplace=True)  # Drop indexes since we're not going to use them

(57, 23)
(3, 23)
(54, 23)


,réservoir source,id source,forme principale source,arbitre,date d'arbitrage,niveau de confiance,commentaire,décision d'alignement,nombre de candidats,score algo max,...,id cible,forme principale cible,type de cible 2,réservoir cible 2,id cible 2,forme principale cible 2,type de cible 3,réservoir cible 3,id cible 3,forme principale cible 3
18,rnv-nz-auth-atc,981023285304302851,"Jones, E.A. (Edward Alexander), 1968-",NaN,NaN,NaN,NaN,auto,1,0.9625,...,104618981,"Jones, Edward Alexander médiéviste 1968-....",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,rnv-nz-auth-atc,981023286040402851,"Vitali, Marco E.",NaN,NaN,NaN,NaN,auto,1,1.0,...,257028161,"Vitali, Marco E. 19..-....",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,rnv-nz-auth-atc,981023286465502851,"Glazovskai︠a︡, M.A",NaN,NaN,NaN,NaN,auto,1,1.0,...,243463812,"Glazovskaâ, Mariâ Al'fredovna 1912-2016",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Chargement des fichiers auxiliaires

Si une comparaison utilisant les notices complètes IdRef et ATC/RNV est demandée, charger ces dernières dans un dataframe.

In [2]:
# Fonctions nécessaires à traiter les fichiers XML provenant d'IdRef (Marc21)

# Chargement des fichiers individuels
def parse_marcxml_file(file_path):
    records = []
    with open(file_path, 'rb') as fh:
        marc_records = parse_xml_to_array(fh)
        for record in marc_records:
            if record != None:
                record_dict = extract_record_data(record)
                if record_dict != None:
                    records.append(record_dict)
    return records

# Extraction des champs nécessaires
def extract_record_data(record):
    field_001 = record.get('001')
    if idref_set == 'personnes':
        field_100 = record.get('100')
        if field_001:
            record_data = {
                'idref_id' : field_001.data.replace('(IDREF)','').strip(),
                '100a': field_100.get('a') if field_100 else None,
                '100d': field_100.get('d') if field_100 else None
            }
        else:
            return None
    else:
        field_110 = record.get('110')
        field_111 = record.get('111')
        if field_001:
            record_data = {
                'idref_id' : field_001.data.replace('(IDREF)','').strip(),
                '110a': field_110.get('a') if field_110 else None,
                '110b': field_110.get('b') if field_110 else None,
                '110g': field_110.get('g') if field_110 else None,
                '111a': field_111.get('a') if field_111 else None,
                '111c': field_111.get('c') if field_111 else None,
                '111d': field_111.get('d') if field_111 else None
            }
    return record_data

# Chargement des fichiers du dossier IdRef
def load_marcxml_files_from_folder(folder_path, limit, status_display):
    all_records = []
    
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.xml'): 
            file_path = os.path.join(folder_path, file_name)
            records = parse_marcxml_file(file_path)
            all_records.extend(records)
        status_display.value += 1
        if (limit > 0 and status_display.value == limit):
            break
    
    # Convert list of records to DataFrame
    df = pd.DataFrame(all_records)
    return df

# Chargement des fichiers auxiliaires si on a choisi le type de validation 'idref-local'
if 'idref-local' in validations:
    
    # Si un fichier CSV a été fourni (l'extraction a déjà été faite), charger les notices depuis ce fichier
    if idref_records_extract and os.path.isfile(idref_records_extract):
        idRef_df = pd.read_csv(idref_records_extract, dtype = str)
        print(f"{len(idRef_df.index)} notices IdRef chargées depuis {idref_records_extract}")
    else:
        # Sinon, on prend une grande respiration et on charge tous les fichiers IdRef dans un dataframe
        
        # Affiche une barre de progression
        if idref_records_limit > 0:
            num_files = idref_records_limit
        else:
            num_files = sum(1 for entry in os.scandir(idref_records_folder) if entry.is_file() and entry.name.endswith('.xml'))
        barre_attente_idRef = IntProgress(min=1, max=num_files, description="Chargement des fichiers IdRef: ", style={'description_width':'initial'},layout={'width':'80%'})
        display(barre_attente_idRef)
        
        idRef_df = load_marcxml_files_from_folder(idref_records_folder, idref_records_limit, barre_attente_idRef)
        
        # Une fois le chargement terminé, on écrit le résultat dans un fichier CSV
        if idref_records_extract:
            idRef_df.to_csv(idref_records_extract, encoding="UTF-8", index=False)
    
        print(f"{len(idRef_df.index)} notices IdRef chargées depuis export OAI IdRef dans {idref_records_folder}")
    
    # Utiliser l'identifiant IdRef comme index (pour accélérer les requêtes)
    idRef_df.set_index('idref_id', inplace=True)
    # Retirer les éventuels doublons
    idRef_df = idRef_df[~idRef_df.index.duplicated(keep='first')]
    
    # Chargement du fichier ATC/RNV
    if atc_records_file_path and os.path.isfile(atc_records_file_path):
        atc_df = pd.read_csv(atc_records_file_path, sep='\t', dtype = str)
        # Extraire le contenu des sous-champs dans des colonnes distinctes
        atc_df['100a'] = atc_df['subfields_content_for_tag2'].str.extract(r'\$\$a ([^,]+, [^$]+)')
        atc_df['100d'] = atc_df['subfields_content_for_tag2'].str.extract(r'\$\$d ([^$]+)')

        # Utiliser l'identifiant comme index (pour accélérer les requêtes)
        atc_df.set_index('id', inplace=True)
        print(f"{len(atc_df.index)} notices ATC/RNV chargées depuis le fichier {atc_records_file_path}")
    else:
        print('Attention! Le fichier des notices ATC/RNV est manquant!')

4293697 notices IdRef chargées depuis input/idref-extrait-personnes-comparaison.csv
121129 notices ATC/RNV chargées depuis le fichier input/20240917_all_ATC_persons_with_AD.tsv


## Définition des fonctions utiles

In [2]:
# Fonction de comparaison des dates
def date_compare(source_date,target_date):
    source_date = str(source_date)
    target_date = str(target_date)
    # Ignorer les points d'interrogation
    source_date = source_date.replace('?','')
    target_date = target_date.replace('?','')
    # Retirer le zéro en début de date
    if source_date.startswith('0'):
        source_date = source_date[1:0]
    if target_date.startswith('0'):
        target_date = target_date[1:0]
    # Si l'une des dates à comparer est vide, on passe
    if ((source_date == '') | (target_date == '')):
        return False
    # Si l'une des dates comporte des points, ne comparer que les chiffres entre eux
    if (('.' in source_date) | ('.' in target_date)):
        #print('Comparaison avec points: ' + source_date + ' et ' + target_date)
        for char in range(0,len(source_date)):
            if ((source_date[char] != '.') & (target_date[char] != '.') & (source_date[char] != target_date[char])):
                #print('Je pense que ' + source_date[char] + ' != ' + target_date[char])
                return True
        return False
    if (source_date != target_date):
        return True
    else:
        return False

# Expression régulière pour trouver les dates
date_pattern = r'(?:ca\.|fl\.)?([0-9\.]{4}|\?)-?([0-9\.]{4}|\?)?'

# Vérification des auteurs-titre

idref_base_url = 'https://idref.fr/'

# Fonction utile pour déterminer si un champ MARC existe
def contains_tag(data, tag_value):
    return any(record['tag'] == tag_value for record in data)



## Moulinette de validation
C'est là que tout se passe!

In [21]:
# Affiche une barre de progression
numrows = df_filtered.shape[0]
barre_attente = IntProgress(min=1, max=numrows, description="État d'avancement: ", style={'description_width':'initial'},layout={'width':'80%'})
display(barre_attente)

if not(output_in_place):
    wrong_dates = []
    wrong_types = []
    misc_errors = []

for index, row in df_filtered.iterrows():
    if not ((row[source_form_colname] == '') | (row[target_form_colname] == '') | pd.isnull(row[source_form_colname]) | pd.isnull(row[target_form_colname])):
        
        # Procéder à la validation en utilisant les notices complètes IdRef et ATC/RNV si demandé
        
        if 'idref-local' in validations:
            # La validation ne peut se faire que si les notices complètes existent
            try:
                notice_idref = idRef_df.loc[row[idref_id_colname]]
                notice_atc = atc_df.loc[row[atc_id_colname]]
                print(notice_idref['100a'])
                print(notice_atc['100a'])
            except KeyError:
                pass
        
        
        
        # Procéder à la validation de date simple seulement si demandé
        if 'dates' in validations:
            source_dates = re.findall(date_pattern, row[source_form_colname])
            target_dates = re.findall(date_pattern, row[target_form_colname])
            if ((len(source_dates) > 0) & (len(target_dates) > 0)):
                if (date_compare(source_dates[0][0],target_dates[0][0]) | date_compare(source_dates[0][1],target_dates[0][1])):
                    # Cas potentiel de dates qui ne correspondent pas
    
                    if (output_in_place):
                        df_filtered.loc[index, output_colname] = "Vérifier dates"
                    else:
                        wrong_dates.append(row)
        
        # Vérifier si la notice IdRef vers laquelle on aligne est de type auteur-titre, seulement si demandé
        if 'auteur-titre' in validations:
            idref_id = row[idref_id_colname]
            url = idref_base_url + str(idref_id) + '.json'
            # Query the IdRef API
            idref_result = requests.get(url)
            if (idref_result.status_code == 200):
                # Identifier si le résultat contient un champ 240
                if contains_tag(idref_result.json()['record']['datafield'], 240):
                    if (output_in_place):
                        df_filtered.loc[index, output_colname] = "Cible de type auteur-titre"
                    else:
                        wrong_types.append(row)
            else:
                # Erreur de requête API
                if (output_in_place):
                    df_filtered.loc[index, output_colname] = "Erreur de requête API IdRef. Code:" + idref_result.status_code
                else:
                    misc_errors.append(row)
    
    else:
        # Il manque un des champs à comparer, autre erreur
        if (output_in_place):
            df_filtered.loc[index, output_colname] = "Erreur de validation"
        else:
            misc_errors.append(row)
    
    barre_attente.value += 1

display(df_filtered.head())

IntProgress(value=1, description="État d'avancement: ", layout=Layout(width='80%'), max=3, min=1, style=Progre…

Jones, Edward Alexander
Jones, E.A. 


,réservoir source,id source,forme principale source,arbitre,date d'arbitrage,niveau de confiance,commentaire,décision d'alignement,nombre de candidats,score algo max,...,id cible,forme principale cible,type de cible 2,réservoir cible 2,id cible 2,forme principale cible 2,type de cible 3,réservoir cible 3,id cible 3,forme principale cible 3
0,rnv-nz-auth-atc,981023285304302851,"Jones, E.A. (Edward Alexander), 1968-",NaN,NaN,NaN,NaN,auto,1,0.9625,...,104618981,"Jones, Edward Alexander médiéviste 1968-....",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,rnv-nz-auth-atc,981023286040402851,"Vitali, Marco E.",NaN,NaN,NaN,NaN,auto,1,1.0,...,257028161,"Vitali, Marco E. 19..-....",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,rnv-nz-auth-atc,981023286465502851,"Glazovskai︠a︡, M.A",NaN,NaN,NaN,NaN,auto,1,1.0,...,243463812,"Glazovskaâ, Mariâ Al'fredovna 1912-2016",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Exports des fichiers de résultats

In [10]:
if not(output_in_place):
    wrong_dates_df = pd.DataFrame(wrong_dates)
    misc_errors_df = pd.DataFrame(misc_errors)

    wrong_dates_df.to_excel(output_file_root + "_dates.xlsx",index=False)
    misc_errors_df.to_excel(output_file_root + "_erreurs.xlsx",index=False)
else:
    print(df_filtered.shape)
    # Combiner le fichier filtré qui a été validé avec les alignements retirés au début
    df_output = pd.concat([df_filtered,df_rest])
    print(df_output.shape)
    df_output.to_excel(output_file_root + "_tout.xlsx",index=False)
    

(41, 24)
(57, 24)
